In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
from tqdm import tqdm
from datetime import datetime
import json
from sklearn.preprocessing import LabelEncoder
import calendar
import spotipy
warnings.filterwarnings("ignore")

# Web scrapping data from Spotify Chart

## Get date needed for each week

format of links to download:
https://spotifycharts.com/regional/global/weekly/2016-12-23--2016-12-30/download
https://spotifycharts.com/regional/global/weekly/2016-12-30--2017-01-06/download

We see that all the dates that appear in the link are Fridays, each being one week apart

In [23]:
#An example of Calendar package
x = []

for month in range(1, 13):
    mycal = calendar.monthcalendar(2025, month)
    week1 = mycal[1]
    week2 = mycal[2]
      
    if week1[calendar.MONDAY] != 0:
        auditday = week1[calendar.MONDAY]
    else: 
        auditday = week2[calendar.MONDAY] 
    print("%10s %2d" % (calendar.month_name[month], auditday))
    x.append("%10s %2d" % (calendar.month_name[month], auditday))

   January  6
  February  3
     March  3
     April  7
       May  5
      June  2
      July  7
    August  4
 September  8
   October  6
  November  3
  December  8


In [4]:
# reference
c = calendar.TextCalendar(calendar.MONDAY)
str1 = c.formatmonth(2017,1)
print(str1)

    January 2017
Mo Tu We Th Fr Sa Su
                   1
 2  3  4  5  6  7  8
 9 10 11 12 13 14 15
16 17 18 19 20 21 22
23 24 25 26 27 28 29
30 31



In [5]:
(calendar.monthcalendar(2017,1)[0][calendar.FRIDAY],calendar.monthcalendar(2017,1)[4][calendar.FRIDAY])
#calendar.monthcalendar(2017,1)[c(0,4)][calendar.FRIDAY]

(0, 27)

In [6]:
#All the fridays in 2017
Fridays17 = []

for month in range(1, 13):
    mycal = calendar.monthcalendar(2017, month)
    
    for week in range(0,len(mycal)):
        if mycal[week][calendar.FRIDAY] != 0:
            #print(2017, month, mycal[week][calendar.FRIDAY])
            Fridays17.append((2017, month, mycal[week][calendar.FRIDAY]))
            
            
Fridays17[0]

(2017, 1, 6)

In [7]:
#All the fridays in 2018
Fridays18 = []

for month in range(1, 13):
    mycal = calendar.monthcalendar(2018, month)
    
    for week in range(0,len(mycal)):
        if mycal[week][calendar.FRIDAY] != 0:
            #print(2018, month, mycal[week][calendar.FRIDAY])
            Fridays18.append((2018, month, mycal[week][calendar.FRIDAY]))


Fridays18[0][2]

5

In [8]:
#fridays in 2019 (Jan to Aug)
Fridays19 = []

for month in range(1, 9):
    mycal = calendar.monthcalendar(2019, month)
    
    for week in range(0,len(mycal)):
        if mycal[week][calendar.FRIDAY] != 0:
            #print(2019, month, mycal[week][calendar.FRIDAY])
            Fridays19.append((2019, month, mycal[week][calendar.FRIDAY]))

Fridays19[2]

(2019, 1, 18)

In [19]:
#Fridays in 2016
Fridays16to19 = [(2016, 12, 23),(2016, 12, 30)]

#Compile all the fridays 2016-2019
Fridays16to19 = Fridays16to19 + Fridays17 + Fridays18 + Fridays19
Fridays16to19[0]

(2016, 12, 23)

In [21]:
#formatting friday dates: 2016-12-23--2016-12-30

Fridays_format = []

for i in range(0, len(Fridays16to19)):
    year = Fridays16to19[i][0]
    month = Fridays16to19[i][1]
    day = Fridays16to19[i][2]  
    
    if month < 10:
        month = str(0)+str(month)
    else:
        month = str(month)
        
    if day < 10:
        day = str(0)+str(day)
    else:
        day = str(day)
    
    #print(str(year)+"-"+month+"-"+day)
    Fridays_format.append(str(year)+"-"+month+"-"+day)
    
Fridays_format[0]

'2016-12-23'

In [22]:
#put all the dates into the format of download link: 
#https://spotifycharts.com/regional/global/weekly/2016-12-23--2016-12-30/download

####################IndexError: list index out of range
# for j in range(0, len(Fridays_format)):
#     str_p1 = "https://spotifycharts.com/regional/global/weekly/" + Fridays_format[j] + "--"
#     if j != len(Fridays_format):
#         print(str_p1 + Fridays_format[j+1] + "/download")
        
# for j in range(0, len(Fridays_format)):
#     print("https://spotifycharts.com/regional/global/weekly/" + Fridays_format[j] 
#           + "--" + Fridays_format[j+1] + "/download")


j = 0
while(j<len(Fridays_format)-1):
    str_p1 = print("https://spotifycharts.com/regional/global/weekly/" + Fridays_format[j] + 
                   "--" + Fridays_format[j+1] + "/download")
    j += 1

https://spotifycharts.com/regional/global/weekly/2016-12-23--2016-12-30/download
https://spotifycharts.com/regional/global/weekly/2016-12-30--2017-01-06/download
https://spotifycharts.com/regional/global/weekly/2017-01-06--2017-01-13/download
https://spotifycharts.com/regional/global/weekly/2017-01-13--2017-01-20/download
https://spotifycharts.com/regional/global/weekly/2017-01-20--2017-01-27/download
https://spotifycharts.com/regional/global/weekly/2017-01-27--2017-02-03/download
https://spotifycharts.com/regional/global/weekly/2017-02-03--2017-02-10/download
https://spotifycharts.com/regional/global/weekly/2017-02-10--2017-02-17/download
https://spotifycharts.com/regional/global/weekly/2017-02-17--2017-02-24/download
https://spotifycharts.com/regional/global/weekly/2017-02-24--2017-03-03/download
https://spotifycharts.com/regional/global/weekly/2017-03-03--2017-03-10/download
https://spotifycharts.com/regional/global/weekly/2017-03-10--2017-03-17/download
https://spotifycharts.com/re

## Billboard Dataset

In [24]:
import csv
import pandas as pd

# billboard_data = pd.read_csv("billboard_lyrics_1964-2015.csv", encoding='utf-8')

my_list =[]
with open('billboard_lyrics_1964-2015.csv', newline='', encoding='utf-8') as file:
    for row in csv.reader(file):
        print(row)
#         my_list.append(row)
    
# my_list
# df = pd.DataFrame(my_list)
# df


['Rank', 'Song', 'Artist', 'Year', 'Lyrics', 'Source']
['1', 'wooly bully', 'sam the sham and the pharaohs', '1965', 'sam the sham miscellaneous wooly bully wooly bully sam the sham  the pharaohs  domingo samudio uno dos one two tres quatro matty told hatty about a thing she saw had two big horns and a wooly jaw wooly bully wooly bully wooly bully wooly bully wooly bully hatty told matty lets dont take no chance lets not belseven come and learn to dance wooly bully wooly bully wooly bully wooly bully wooly bully matty told hatty thats the thing to do get you someone really to pull the wool with you wooly bully wooly bully wooly bully wooly bully wooly bully lseven  the letter l and the number 7 when typed they form a rough square l7 so the lyrics mean lets not be square', '3']
['2', 'i cant help myself sugar pie honey bunch', 'four tops', '1965', ' sugar pie honey bunch you know that i love you i cant help myself i love you and nobody elsein and out my life you come and you go leaving 

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xcc in position 5859: invalid continuation byte

In [59]:
billboard_df = pd.read_csv("billboard_lyrics_1964-2015.csv",
                   encoding='latin-1')
billboard_df

#billboard_df.describe()

,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0
5,6,downtown,petula clark,1965,when youre alone and life is making you lonel...,1.0
6,7,help,the beatles,1965,help i need somebody help not just anybody hel...,3.0
7,8,cant you hear my heart beat,hermans hermits,1965,carterlewis every time i see you lookin my way...,5.0
8,9,crying in the chapel,elvis presley,1965,you saw me crying in the chapel the tears i s...,1.0
9,10,my girl,the temptations,1965,ive got sunshine on a cloudy day when its cold...,3.0


In [53]:
billboard_df.iloc[[4],[4]]

,Lyrics
4,you never close your eyes anymore when i kiss...


In [31]:
billboard_df

,Rank,Year,Source
count,5100.0000,5100.000000,4913.000000
mean,50.5000,1990.000000,1.400977
std,28.8689,14.721045,0.890375
min,1.0000,1965.000000,1.000000
25%,25.7500,1977.000000,1.000000
50%,50.5000,1990.000000,1.000000
75%,75.2500,2003.000000,1.000000
max,100.0000,2015.000000,5.000000


## spotipy

In [76]:
#setting up spotipy
import sys
import spotipy
import spotipy.util as util

scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print ("Usage: %s username" % (sys.argv[0],))
    sys.exit()
    
token = util.prompt_for_user_token(username,scope,client_id='ff9ff248e80e428bbb9796e1a7d62aeb',
                                   client_secret='7f70a7c884494c6c96d2e6f03c4d2388', redirect_uri='http://localhost/')



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=ff9ff248e80e428bbb9796e1a7d62aeb&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2F&scope=user-library-read in your browser


Enter the URL you were redirected to: http://localhost/?code=AQCmsq9d26p649wcnAJMcjgYtEDzpJPYNjbOnRmGdRoZTYwkI72heeQCuaQ5VXcH1Qr8Vf5fAAY--j7vpE3keTUxE2iWwr13Gk8yS1N-Y3N90xtuEJvYAfIM_1Ho5RES-bha4hXknvK-zgg2FTrt0vm21rjFARe2Jz6z4E39u5rJpW9eZ2ugsMJLYd-V7Jw2xX5bZfSVa9l0aGCJ




In [77]:
sp = spotipy.Spotify(auth=token)

In [78]:
name = 'Shawn Mendes'

results = sp.search(q='artist:' + name, type='artist')
print(results)

{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3AShawn+Mendes&type=artist&market=US&offset=0&limit=10', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7n2wHs1TKAczGzO7Dd2rGr'}, 'followers': {'href': None, 'total': 19751691}, 'genres': ['canadian pop', 'pop', 'viral pop'], 'href': 'https://api.spotify.com/v1/artists/7n2wHs1TKAczGzO7Dd2rGr', 'id': '7n2wHs1TKAczGzO7Dd2rGr', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/6bd59cfbd3e1e6394af710bddcd62999aae31c6f', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/3e0c03d2ef76a80913734e90cf466f9e1478e88c', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/4ff072e64ddd4098ec1f089fb80839b396a19505', 'width': 160}], 'name': 'Shawn Mendes', 'popularity': 96, 'type': 'artist', 'uri': 'spotify:artist:7n2wHs1TKAczGzO7Dd2rGr'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/4pjjRVF9bZ8y9iXIuVgOam'}, 'followers': {'href': None, 'total': 1438}, 'genre